<a href="https://colab.research.google.com/github/DhawaDong/Breast-Cancer-ANN/blob/main/ANN_BreastCancerDiagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd;
from sklearn.model_selection import train_test_split;
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
DataSetBC = pd.read_csv(r'/content/drive/MyDrive/7#MachineLearning#Projects/BreastCancerDataSet.csv')

In [4]:
DataSetBC.shape

(569, 33)

In [5]:
DataSetBC.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [6]:
x = DataSetBC.drop(['id', 'diagnosis', 'Unnamed: 32'], axis =1);   #pandas DataFrame
y = DataSetBC['diagnosis'];   # pandas series.

In [7]:
DiagClass = {'M': 1.0, 'B': 0.0}
y = y.map(DiagClass);

In [8]:
x.shape, y.shape

((569, 30), (569,))

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [10]:
# Standardize features
scaler = StandardScaler()     #normalization for similar ranged mean and variance
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

#y_test_array = np.array(y_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [12]:

# Define the ANN model
class BreastCancerANN(nn.Module):
    def __init__(self):
        super(BreastCancerANN, self).__init__()   # calling constructor
        self.layer1 = nn.Linear(30, 16)  # 30 input features, 16 hidden units
        self.layer2 = nn.Linear(16, 1)   # 16 hidden units, 1 output unit
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

In [13]:
# Instantiate the model
model = BreastCancerANN()

In [14]:
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  #SGD, Adagrad, RMSprop, Adadelta

In [15]:
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.view(-1, 1))  #evaluate the loss value.

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()    #backpropagation, gradient of parameres(w) wrt error or loss.
    optimizer.step()   #adjust the new parameter values.

    # Print the loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/1000], Loss: 0.6436
Epoch [20/1000], Loss: 0.5611
Epoch [30/1000], Loss: 0.4902
Epoch [40/1000], Loss: 0.4296
Epoch [50/1000], Loss: 0.3781
Epoch [60/1000], Loss: 0.3344
Epoch [70/1000], Loss: 0.2974
Epoch [80/1000], Loss: 0.2662
Epoch [90/1000], Loss: 0.2399
Epoch [100/1000], Loss: 0.2179
Epoch [110/1000], Loss: 0.1992
Epoch [120/1000], Loss: 0.1833
Epoch [130/1000], Loss: 0.1697
Epoch [140/1000], Loss: 0.1580
Epoch [150/1000], Loss: 0.1479
Epoch [160/1000], Loss: 0.1390
Epoch [170/1000], Loss: 0.1311
Epoch [180/1000], Loss: 0.1242
Epoch [190/1000], Loss: 0.1182
Epoch [200/1000], Loss: 0.1127
Epoch [210/1000], Loss: 0.1078
Epoch [220/1000], Loss: 0.1034
Epoch [230/1000], Loss: 0.0994
Epoch [240/1000], Loss: 0.0958
Epoch [250/1000], Loss: 0.0925
Epoch [260/1000], Loss: 0.0895
Epoch [270/1000], Loss: 0.0868
Epoch [280/1000], Loss: 0.0843
Epoch [290/1000], Loss: 0.0819
Epoch [300/1000], Loss: 0.0798
Epoch [310/1000], Loss: 0.0778
Epoch [320/1000], Loss: 0.0759
Epoch [330/1000],

In [16]:
# Evaluate the model on the test set
with torch.no_grad():
    model.eval()    #set the model in evaluation mode
    predictions = model(X_test_tensor)
    predictions = (predictions >= 0.5).float()  # Convert probabilities to binary predictions
    accuracy = torch.sum(predictions == y_test_tensor.view(-1, 1)).item() / y_test_tensor.size(0)

In [17]:
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.9825
